# Building a Basic AI Agent with LangChain

In this tutorial, we will create a simple AI agent that can decide between three tools based on user queries:

- A **Calculator** tool for math questions.
- A **Calendar** tool to add or check events.
- A **Knowledge** tool to answer factual questions.

We will use the Hugging Face pipeline for language model interactions and implement the agent logic using LangChain.

Let's get started by setting up the environment and installing the required libraries.


In [ ]:
# Install required packages
!pip install langchain transformers huggingface_hub langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


### Importing Libraries and Setting Up the Hugging Face Pipeline

In this step, we import the necessary libraries and initialize a Hugging Face pipeline for text generation.

Since we are building a basic agent, we'll use a small model like `distilgpt2` to keep things fast and lightweight.

Later, we'll define the three tools and the agent logic.


In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import re

In [ ]:
hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=100,
    do_sample=False,
    top_p=1.0,
    repetition_penalty=1.0,
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
# Wrap the pipeline for LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

### Defining the Tools

We will create three simple tools:

1. **Calculator Tool**: Evaluates basic math expressions.
2. **Calendar Tool**: Stores and retrieves events in a simple in-memory calendar.
3. **Knowledge Tool**: Answers factual questions using the language model itself.

Each tool will be wrapped as a LangChain `Tool` object with a name and a function.

Let's define these tools now.


In [ ]:
# Simple in-memory calendar storage
calendar_events = {}

In [ ]:
def calculator_tool(query: str) -> str:
    try:
        # Evaluate simple math expressions safely
        result = str(eval(query, {"__builtins__": {}}))
    except Exception:
        result = "Sorry, I couldn't calculate that."
    return result

In [ ]:
def calendar_tool(query: str) -> str:
    # Very basic parser for add/check commands
    query = query.lower()
    if "add event" in query:
        # Extract event name and date (very simple parsing)
        parts = query.split("add event ")[1].split(" on ")
        if len(parts) == 2:
            event, date = parts
            calendar_events[date.strip()] = event.strip()
            return f"Event '{event.strip()}' added on {date.strip()}."
        else:
            return "Please specify event and date in format: add event [event] on [date]."
    elif "check event" in query:
        date = query.split("check event ")[1].strip()
        event = calendar_events.get(date, "No events found on this date.")
        return event
    else:
        return "I can add events with 'add event [event] on [date]' or check events with 'check event [date]'."

In [ ]:
def knowledge_tool(query: str) -> str:
    # Use the language model directly for general questions
    response = llm(query)
    return response['text'].strip()

In [ ]:
# Wrap them as LangChain Tools
tools = [
    Tool(name="Calculator", func=calculator_tool, description="Useful for math calculations."),
    Tool(name="Calendar", func=calendar_tool, description="Add or check calendar events."),
    Tool(name="Knowledge", func=knowledge_tool, description="Answer general knowledge questions."),
]

### Explanation of Tools

- **Calculator Tool:** Uses Python's `eval` safely (with no builtins) to compute simple math expressions. If evaluation fails, it returns an error message.

- **Calendar Tool:** Maintains an in-memory dictionary of events. It can:
  - Add events with the command format `"add event [event] on [date]"`.
  - Check events on a date with `"check event [date]"`.

- **Knowledge Tool:** Sends the query directly to the language model (`llm`) for general knowledge or open-ended questions.

Each tool is wrapped using LangChain's `Tool` class, which requires a name, a function, and a short description.

Next, we will implement the AI agent that decides which tool to call based on the user's query.


### Building a Prompt-Based AI Agent

Instead of hardcoding decision rules, we'll leverage the language model to decide which tool to use.

We will:

- Create a prompt template that explains the tools and instructs the LLM to choose one.
- Parse the LLM’s output to extract the tool name and input.
- Call the selected tool with the extracted input.
- Return the tool’s response as the agent’s answer.

This approach lets the AI agent dynamically choose tools based on natural language understanding.

Let's implement this now.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.chains import LLMChain

In [ ]:
class ToolNameParser(AgentOutputParser):
    def parse(self, llm_output: str):
        tool_name = llm_output.strip().capitalize()
        print(f"LLM Output for ToolNameParser: '{llm_output}'")  # Debug
        if tool_name in ["Calculator", "Calendar", "Knowledge"]:
            return AgentAction(tool_name, "", llm_output)
        else:
            print("Parsing failed.")  # Debug
            return AgentFinish({"output": llm_output}, llm_output)


In [ ]:
prompt_template = """


 """

In [ ]:
prompt = PromptTemplate(template=prompt_template, input_variables=["input"], partial_variables={"tool_descriptions": "\n".join([f"{tool.name}: {tool.description}" for tool in tools])})

In [ ]:
# Create LLMChain with our prompt and LLM
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=ToolNameParser(),
    stop=["\n"],  # Stop generation after first line
    allowed_tools=[tool.name for tool in tools],
)




In [ ]:
# Wrap in AgentExecutor for running
agent_executor = AgentExecutor(agent=agent, tools=tools, max_iterations=1)

### Running the Prompt-Based AI Agent

We have now created an `LLMChain` combining our language model and the prompt template.

This is passed to the `LLMSingleActionAgent` which uses the LLM output to decide the tool and input dynamically.

Finally, we wrap this agent in an `AgentExecutor` that handles calling the tools and returning the response.

You can run the agent on user queries using:

```python
agent_executor.run("your query here")


In [ ]:
queries = [
    "15 + 27",
    "Add event team meeting on 2025-07-20",
    "Check event 2025-07-20",
    "Who is the president of the United States?",
]

In [ ]:
def run_agent_and_invoke_tool(query: str):
    # Run agent to get the tool name decision
    agent_output = agent_executor.agent.llm_chain.predict(input=query)

    tool_name = agent_output.strip().capitalize()
    print(f"Agent decided tool: {tool_name}")

    # Find the matching tool
    matched_tool = next((t for t in tools if t.name == tool_name), None)
    if not matched_tool:
        return f"No matching tool found for: {tool_name}"

    # Call the tool function with the original query (or customize input if needed)
    result = matched_tool.func(query)
    return f"Tool: {tool_name}\nResult: {result}"

# Run test queries
for q in queries:
    print(f"Query: {q}")
    print(run_agent_and_invoke_tool(q))
    print("-" * 40)


Query: 15 + 27
Agent decided tool: Calculator
Tool: Calculator
Result: 42
----------------------------------------
Query: Add event team meeting on 2025-07-20
Agent decided tool: Calendar
Tool: Calendar
Result: Event 'team meeting' added on 2025-07-20.
----------------------------------------
Query: Check event 2025-07-20
Agent decided tool: Calendar
Tool: Calendar
Result: team meeting
----------------------------------------
Query: Who is the president of the United States?
Agent decided tool: George w. bush
No matching tool found for: George w. bush
----------------------------------------
